<a href="https://colab.research.google.com/github/daisuke-8080/kaggle_twitter_disaster_compe_with_BERT/blob/main/twitter_disaster_competition_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 8.3 MB/s 
     |████████████████████████████████| 831.4 MB 2.8 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.9.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.9.0 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import torch

In [ ]:
import json 
import os

In [ ]:
!pip install transformers
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 16.0 MB/s 
     |████████████████████████████████| 101 kB 10.7 MB/s 
     |████████████████████████████████| 6.6 MB 51.3 MB/s 
     |████████████████████████████████| 596 kB 42.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Google Colaboratory上からGoogle Drive上のファイルにアクセスできるようにする
from google.colab import drive
drive.mount('/content/drive')

!mkdir ~/.kaggle
!touch /root/.kaggle/kaggle.json
# kaggle.jsonファイルの内容を次の行にコピーしjsonファイルに書き写す
api_token = {"username":"oikawadaisuke","key":"e3ce4f5964b7059090bf86d85bd62b4b"}
with open('/root/.kaggle/kaggle.json', 'w') as file:
 json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle competitions download -c nlp-getting-started #グーグルドライブに保存するならコメントを外して→を実行 -p /content/drive/MyDrive/kaggle/'ここにコンペネームを貼りつける'

  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 92.7MB/s]


In [ ]:
!unzip '/content/nlp-getting-started.zip'

Archive:  /content/nlp-getting-started.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
sample_df = pd.read_csv('/content/sample_submission.csv')
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

In [ ]:
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [ ]:
train_df.fillna('nan', inplace=True)

for i in range(len(train_df)):
  ex = train_df['text'][i]
  ex_keyword = train_df['keyword'][i]
  ex_location = train_df['location'][i]
  ex += ' ' + '[SEP]' + ' ' + ex_keyword + ' ' + ex_location
  train_df.iloc[i, 3] = ex
del train_df['id']
del train_df['keyword']
del train_df['location']
train_df.to_csv('/content/train_fixed.csv', index=False)

In [ ]:
test_df.fillna('nan', inplace=True)

for i in range(len(test_df)):
  ex = test_df['text'][i]
  ex_keyword = test_df['keyword'][i]
  ex_location = test_df['location'][i]
  ex += ' ' + '[SEP]' + ' ' + ex_keyword + ' ' + ex_location
  test_df.iloc[i, 3] = ex
del test_df['id']
del test_df['keyword']
del test_df['location']
test_df.to_csv('/content/test_fixed.csv', index=False)

In [ ]:
import pkg_resources

pkglist = ['torch', 'torchtext']
for dist in pkg_resources.working_set:
    if dist.project_name in pkglist:
        print(dist.project_name, dist.version)

torchtext 0.10.0
torch 1.9.0


In [ ]:
def my_tokenize(sentence):
  tokens = tokenizer.tokenize(sentence)
  tokens  = tokens[:252]
  return tokens

In [ ]:
import torchtext
from torchtext.legacy import data

TEXT = data.Field(batch_first=True,
                  use_vocab=False,
                  tokenize=my_tokenize,
                  preprocessing=tokenizer.convert_tokens_to_ids,
                  init_token=tokenizer.cls_token_id,
                  pad_token=tokenizer.pad_token_id,
                  fix_length=256)

LABEL = data.Field(sequential=False, use_vocab=False)

In [ ]:
from torchtext.legacy.data import TabularDataset

train_dataset = TabularDataset(path='/content/train_fixed.csv',
                                                    format='csv',
                                                    skip_header=True,
                                                    fields=[('Text',TEXT) , ('Label',LABEL)])

In [ ]:
from torchtext.legacy.data import TabularDataset

test_dataset = TabularDataset(path='/content/test_fixed.csv',
                                                    format='csv',
                                                    skip_header=True,
                                                    fields=[('Text',TEXT)])

In [ ]:
from torchtext.legacy.data import Iterator
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
batch_size = 16
train_loader = Iterator(train_dataset, batch_size=16, 
                        train=True,
                        shuffle=True,
                        device=device)

In [ ]:
test_loader = Iterator(test_dataset, batch_size=16,
                       train=False,
                       sort=False,
                       device=device)

In [ ]:
from transformers import BertModel
bert = BertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import torch.nn as nn

class BertDisaster(nn.Module):
  def __init__(self, bert, output_dim):
    super().__init__()

    self.bert = bert
    linear_embedding_dim = bert.config.to_dict()['hidden_size']
    self.linear = nn.Linear(linear_embedding_dim, output_dim)

  def forward(self, input_text):
    embeded = self.bert(input_text)[1]
    output = self.linear(embeded)
    
    return output

In [ ]:
output_class_dim = 2
model = BertDisaster(bert, output_class_dim).to(device)

In [ ]:
import torch.optim as ouptim
from transformers import AdamW, get_constant_schedule_with_warmup

optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-6)
criterion = nn.CrossEntropyLoss().to(device)

def get_scheduler(optimizer, warmup_steps):
  scheduler = get_constant_schedule_with_warmup(optimizer, warmup_steps)
  return scheduler

def categorical_accuracy(preds, y):
  max_preds = preds.argmax(dim=1, keepdim=True)
  correct_num = (max_preds.squeeze(1)==y).float()
  accuracy = correct_num.sum() / len(y)
  return accuracy

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from tqdm.notebook import tqdm

def train_bert(model, iterator, optimizer, criterion, scheduler):
  
  model.train()
  epoch_loss = 0
  epoch_acc = 0

  for batch in tqdm(iterator):
    optimizer.zero_grad()
    torch.cuda.empty_cache()

    text = batch.Text
    label = batch.Label

    predictions = model(text)

    loss = criterion(predictions, label)
    acc = categorical_accuracy(predictions, label)

    loss.backward()

    scheduler.step()
    optimizer.step()

    epoch_loss += loss.item()
    epoch_acc += acc.item()

    text.detach().to('cpu')
    label.detach().to('cpu')
    
  average_loss = epoch_loss / len(iterator)
  average_acc = epoch_acc / len(iterator)

  return average_loss, average_acc

In [ ]:
import time

def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins*60))
  return elapsed_mins, elapsed_secs

In [ ]:
import math

num_epochs = 5
train_len = len(train_df)

warmup_percent = 0.2
total_steps = math.ceil(num_epochs*train_len*1. / batch_size)
warmup_steps = int(total_steps*warmup_percent)

scheduler = get_scheduler(optimizer, warmup_steps)

In [ ]:
for epoch in range(num_epochs):
  start_time = time.time()

  train_loss, train_acc = train_bert(model, train_loader, optimizer, criterion, scheduler)

  end_time = time.time()

  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')

  0%|          | 0/476 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch: 01 | Epoch Time: 6m 57s
	Train Loss: 0.513 | Train Acc: 74.44%


  0%|          | 0/476 [00:00<?, ?it/s]

Epoch: 02 | Epoch Time: 7m 15s
	Train Loss: 0.379 | Train Acc: 84.19%


...

Epoch: 03 | Epoch Time: 7m 15s
	Train Loss: 0.295 | Train Acc: 88.68%


  0%|          | 0/476 [00:00<?, ?it/s]

Epoch: 04 | Epoch Time: 7m 14s
	Train Loss: 0.214 | Train Acc: 92.53%


  0%|          | 0/476 [00:00<?, ?it/s]

Epoch: 05 | Epoch Time: 7m 15s
	Train Loss: 0.161 | Train Acc: 94.50%


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/kaggle/bert-nli_epoch_6.pt')

In [ ]:
weight = torch.load('/content/drive/MyDrive/kaggle/bert-nli.pt', map_location=device)
model.load_state_dict(weight)

<All keys matched successfully>

In [ ]:
for batch in tqdm(train_loader):
  torch.cuda.empty_cache()
  text = batch.Text
  label = batch.Label

  text.detach().to('cpu')
  label.detach().to('cpu')

  0%|          | 0/476 [00:00<?, ?it/s]

In [ ]:
from tqdm.notebook import tqdm

base = torch.zeros((1), device=device)

model.eval()

for test_batch in tqdm(test_loader):

  batch_text = test_batch.Text

  batch_predictions = model(batch_text)

  batch_predictions = batch_predictions.argmax(dim=1)

  base = torch.cat((base, batch_predictions), dim=0)

test_prediction = base[1:]

print(test_prediction)

  0%|          | 0/204 [00:00<?, ?it/s]

tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:0')


In [ ]:
test_prediction = test_prediction.to('cpu').detach().numpy().copy()
test_prediction = test_prediction.astype(int)
submission_df = sample_df.copy()
submission_df['target'] = test_prediction
submission_df.to_csv('/content/BERT_prediction_epoch_5.csv', index=False)